## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating libor rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* 3 sets of forward libor rates and their fixing dates, one for the Libor 1M, one for the Libor 3M and one for the Libor 6M.
* The static data of the pool of swaps (nominal, maturity, tenor of the first leg, tenor of the second leg)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

In [1]:
from datetime import date
today = date(2019, 10, 31)

# ois_quotes.csv
# euribor1M_proj1.csv
# euribor3M_proj1.csv
# euribor6M_proj1.csv
# basis_swaps_proj1.csv